In [5]:
import os
import numpy as np
from collections import Counter
#from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
import pandas as pd

In [6]:
# third trail

def words_generator(fileobj):
    for line in fileobj:
        for word in line.split():
            yield word

def make_Dictionary(root_dir):
    word_count_dict={}
    file_dict = {}
    emails_dirs = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]    
    for emails_dir in emails_dirs:
        dirs = [os.path.join(emails_dir,f) for f in os.listdir(emails_dir)]
        for d in dirs:
            emails = [os.path.join(d,f) for f in os.listdir(d)]
            
            for mail in emails:
                file_name = mail.rsplit('/', 1)[-1]
                #file_name = file_name.replace('.','')
                file_dict[file_name] = {}
                if mail.split(".")[-2] == 'spam':
                    file_dict[file_name]["is_spam"] = 1 #spam
                else:
                    file_dict[file_name]["is_spam"] = 0 #ham
                    
                f = open(mail,"r")
                words = words_generator(f)
                for word in words:
                    if word.isalpha()==False:
                        continue
                        
                    if word not in word_count_dict:
                          word_count_dict[word] = 0
                    word_count_dict[word]+= 1  
                    
                    if word not in file_dict[file_name]:
                        file_dict[file_name][word] = 0
                    file_dict[file_name][word]+=1
                    
    return word_count_dict,file_dict


In [7]:
    def find_most_frequent_words(word_count_dict):
        #word_freq_dict = {}
        df = pd.DataFrame(word_count_dict.items(), columns=['word', 'total_freq'])
        #df['total_freq'] = df['freq_values'].apply(lambda x: x.get('total'))
        print (len(df))
        df = df.sort(['total_freq'],ascending=False).head(n=3000)
        return df['word'].tolist()
    
    

In [8]:
def get_freq(word,freq_dict):
    if word in freq_dict:
        return freq_dict[word]
    else:
        return 0

In [9]:
import os.path
fname = "/home/datascience/Desktop/SpamFiltering/features.csv"

def extract_features(file_dict,freq_words): 
    features_df = pd.DataFrame(file_dict.items(), columns=['file_name', 'freq_values'])
    features_df['is_spam'] = features_df['freq_values'].apply(lambda x: x.get('is_spam'))
    for word in freq_words:
            features_df[word] = features_df['freq_values'].apply(lambda freq_dict: get_freq(word,freq_dict))
    columns = list(features_df.columns.values)
    columns.remove('freq_values')
    if os.path.isfile(fname):
        with open(fname, 'a') as f:
            features_df.to_csv(f, columns =columns, header=False)
    else:
        features_df.to_csv(fname, columns = columns,header=columns, index=None,sep='\t')
    del features_df


In [11]:
main_dir = "C:\\Users\\sarra\\Downloads\\RESEARCH\\SpamFiltering-master\\SpamFiltering-master\\enronEmailData"

word_count_dict,file_dict = make_Dictionary(main_dir)

IndexError: list index out of range

In [72]:
freq_words = find_most_frequent_words(word_count_dict)

142102


In [79]:
from itertools import islice

def chunks(data, SIZE):
    it = iter(data)
    for i in xrange(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}
        
chunks_list = []
for item in chunks(file_dict, 30):
    extract_features(item,freq_words)
    

In [67]:
len(chunks_list)

1124

In [64]:
features_df = extract_features(file_dict,freq_words)

MemoryError: 